Before we do any work, we need to import several functions from cdapython:
- `Q` and `query` which power the search
- `columns` which lets us view entity field names
- `unique_terms` which lets view entity field contents

We're also asking cdapython to report it's version so we can be sure we're using the one we mean to.

In [ ]:
from cdapython import Q, columns, unique_terms,query, constantVariables
import cdapython
print(cdapython.__file__)
print(cdapython.__version__)
integration_host = "http://35.192.60.10:8080/"
Q.set_host_url(integration_host)

CDA data comes from three sources:
- The [Proteomic Data Commons](https://proteomic.datacommons.cancer.gov/pdc/) (PDC)
- The [Genomic Data Commons](https://gdc.cancer.gov/) (GDC)
- The [Imaging Data Commons](https://datacommons.cancer.gov/repository/imaging-data-commons) (IDC)

The CDA makes this data searchable in four main endpoints:

- `subject`: Specific, unique, individuals
- `researchsubject`: Study-individual aggregate entities. A Subject who was part of three studies will appear as three ResearchSubjects
- `specimen`: Samples taken from individual
- `file`: Data about Subjects, ResearchSubjects, Specimens, and their associated information

and two endpoints that offer deeper information about data in the researchsubject endpoint:

- `diagnosis`: Information about what medical diagnosis a researchsubject has
- `treatment`: Information about what medical treatment(s) were performed for a given diagnosis

If you are looking to build a cohort of distinct individuals who meet some criteria, search by `subject`. If you want to build a cohort, but are particularly interested in studies rather than the participates per se, search by `researchsubject`. If you are looking for biosamples that can be ordered or a specfic format of information (for e.g. histological slides) start with `specimen`. If you are primarily looking for files you can reuse for your own analysis, start with `file`.

In CDA search, these concepts can also be chained together, so you can look specifically for specimen subjects, or researchsubject diagnoses. In the four 'main' tables, all of the rows will have one or more files associated with them that can be directly found by chaining, as in specimen files. Diagnosis and treatment do not have files directly associated with them and so can only be used to find files in conjunction with the other searches.

In all cases, any search can use any metadata field, the only difference between search types is what type of data you return by default. 



## Basic search with endpoints

Let's try a broad search of the CDA to see what information exists about cancers that were first diagnosed in the brain. To run this simple search, we would first construct a query in `Q` and save it to a variable `myquery`:

In [ ]:
myquery = Q('ResearchSubject.primary_diagnosis_site = "brain"')


<div style="background-color:#6ce6b9;color:black;padding:20px;">
<h3>Where did those terms come from?</h3>
    
If you aren't sure how we knew what terms to put in our search, please refer back to the <a href="../SearchTerms">What search terms are available?</a> notebook. 
</div>

### subject
Now we can use that query to search any of information types. Let's start by looking at what Subjects meet our criteria. To do that, we will send our query to the subject endpoint, then ask for it to run:

In [ ]:
subjectresults = myquery.subject.run()

We saved the output in a variable `subjectresults`, so we don't get much visible output. To see what our results are, we need to look into the variable. The simplest way is to call `subjectresults` directly:

In [ ]:
subjectresults

This output tells us our QueryID, which we don't really need, but the computer does to track our questions. Then it tells us four parameters that describe our results:

---

- **Offset:** This is how many rows of information we've told the query to skip in the data, here we didn't tell it to skip anything, so the offset is zero
- **Count:** This is how many rows the current page of our results table has. To keep searches fast, we default to pages with 100 rows.
- **Total Row Count:** This is how many rows are in the full results table
- **More pages:** This is always a True or False. False means that our current page has all the available results. True means that we will see only the first 100 results in this table, and will need to page through for more.

---
    
Now that we've seen the metadata about our results, let's look at the actual table. The easiest way to do this is by using the python function `.to_dataframe()` on our `subjectresults` variable:

In [ ]:
subjectresults.to_dataframe()

By default `to_dataframe()` shows us the first and last five rows for the first page of our results, so we can easily preview our data.

Since we queried the Subject endpoint, our default results tell us Subject level information, that is, information about unique individuals: their sex, race, age, species, etc. The `id` column tells us the unique identifier for each individual. The identifier column has nested information about what study or studies a Subject participated in, and will list all of their researchsubject identifiers. 

The `to_dataframe()` function converts the results to a pandas dataframe. So if we save the dataframe to a variable, we can use any pandas functions to explore it. For example, lets see whether any of the Subjects in our first 100 results are black or african american. First we'll save the results to a dataframe, then subset that dataframe to only show rows where the word "black" appears in the "race" column. "NAs" which are shown as "None" in these tables, so for our filter to work, we'll need to specifically tell it to ignore NAs. We're also telling it we want the word "black" regardless of capitalization with `case=False`:


In [ ]:
subjectdata = subjectresults.to_dataframe()
subjectdata[subjectdata['race'].str.contains("black", case=False, na=False)]

There are three subjects in our first hundred results that meet the criteria. If we just want to be sure that the data contains some value, this might be good enough. But often we want to search the entire set of results and not just the first page. 

We'll cover how to work with large results dataframes in the [Pagination](../Pagination.ipynb) notebook. Or, learn how to get summary information from search results in the [Data Summaries](../DataSummaries.ipynb) notebook.


---

<div style="background-color:#a2f2ed;color:black;padding:20px;">

<h3>Subject Field Definitions</h3>

<i>A subject is a specific, unique individual: for e.g. a single human. When consent allows, a given entity will have a single subject ID that can be connected to all their studies and data across all datasets</i>

    
<ul>
  <li><b>id:</b> The unique identifier for this subject</li>
  <li><b>identifier:</b> An embedded array of information that includes the originating data center and the ID the subject had there</li>
  <li><b>species:</b> The species of the subject</li>
  <li><b>sex:</b> A reference to the biological sex of the donor organism. </li>
  <li><b>race:</b> The race of the subject</li>
  <li><b>ethnicity:</b> The ethnicity of the subject</li>
  <li><b>days_to_birth:</b> Number of days between the date used for index and the date from a person's date of birth represented as a calculated negative number of days</li>
  <li><b>subject_associated_project:</b> An embedded array of the names of projects (studies) the subject was part of</li>
  <li><b>vital_status:</b> Whether the subject is alive</li>
  <li><b>age_at_death:</b> The number of days after first enrollment that the subject died</li>
  <li><b>cause_of_death:</b> The cause of death, if known</li>
</ul>  

</div>
    
---

### researchsubject

If we're interested in what researchsubjects meet our critera, we can also run our query against the researchsubject endpoint:

In [ ]:
researchsubjectresults = myquery.researchsubject.run()
researchsubjectresults

Now we see that our 2314 subjects have 2923 researchsubjects between them, that means that some, but not all, of our subjects were participants in more than one study. Let's peek at the data:

In [ ]:
researchsubjectresults.to_dataframe()

Each row from the researchsubject endpoint results tells us about a subject in a given study. Using this endpoint we can find out information like what studies fit our search criteria, and also get data that we can filter to have only subjects from multiple studies, or only subjects from single studies.

Any given subject will have one row per study they participated in. The subject_id in the last column of this view is the same as the `id` in the first column of the Subjects endpoint results. You can use this to combine information across endpoints, which is covered in the [Merging Results](../MergingResults.ipynb) notebook.


---

<div style="background-color:#a2f2ed;color:black;padding:20px;">

<h3>ResearchSubject Field Definitions</h3>

<i>A research subject is the entity of interest in a research study, typically a human being or an animal, but can also be a device, group of humans or animals, or a tissue sample. Human research subjects are usually not traceable to a particular person to protect the subject’s privacy. An individual who participates in 3 studies will have 3 researchsubject IDs</i>
    
<ul>
  <li><b>id:</b> The unique identifier for this researchsubject</li>
  <li><b>identifier:</b> An embedded array of information that includes the originating data center and the ID the researchsubject had there</li>
  <li><b>member_of_research_project:</b> The name of the study/project that the subject particpated in</li>
  <li><b>primary_diagnosis_condition:</b> The cancer, disease or other condition under study</li>
  <li><b>primary_diagnosis_site:</b> The primary_disease_site that qualifies the researchsubject for the research_project</li>
  <li><b>subject_id:</b> An identifier for the subject. Can be joined to the `id` field from subject results</li>
</ul>  

</div>
    
---

### diagnosis

The diagnosis endpoint is an extension of the researchsubject endpoint, and returns information about research subjects that have a diagnosis that meets our search criteria:

In [ ]:
diagnosisresults = myquery.diagnosis.run()
diagnosisresults.to_dataframe()


---

<div style="background-color:#a2f2ed;color:black;padding:20px;">

<h3>Diagnosis Field Definitions</h3>

<i>A diagnosis is a medical classification of a disease for a given research subject in a given study. A single research subject may have different diagnoses across different studies</i>

    
<ul>
  <li><b>id:</b> The unique identifier for this diagnosis in this research subject</li>
  <li><b>identifier:</b> An embedded array of information that includes the originating data center and the ID the diagnosed researchsubject had there</li>
  <li><b>primary_diagnosis:</b> The main medical diagnosis for this subject in this study</li>
  <li><b>age_at_diagnosis:</b> The subjects age in days after birth on the day they were first diagnosed</li>
  <li><b>morphology:</b> The <a href="https://www.who.int/standards/classifications/other-classifications/international-classification-of-diseases-for-oncology">International Classification of Diseases for Oncology</a> diagnosic code for this diagnosis</li>
  <li><b>stage:</b> A measure of disease spread. Different diseases may use different staging criteria, please refer to the originating data source to see what staging system is reported</li>
  <li><b>grade:</b> A measure of cell abnormality. Different diseases may use different grading criteria, please refer to the originating data source to see what grading system is reported</li>
  <li><b>method_of_diagnosis:</b> The test or system used for determining the diagnosis</li>
  <li><b>subject_id:</b> An identifier for the subject. Can be joined to the `id` field from subject results</li>
  <li><b>researchsubject_id:</b> An identifier for the subject. Can be joined to the `id` field from researchsubject results</li>
</ul>  

</div>
    
---




### treatment

The treatment endpoint is an extension of diagnosis and returns information about treatments undertaken on research subjects that have a given diagnosis that meets our search criteria:

In [ ]:
treatmentresults = myquery.treatment.run()
treatmentresults.to_dataframe()


---

<div style="background-color:#a2f2ed;color:black;padding:20px;">

<h3>Treatment Field Definitions</h3>

<i><i>A treatment is a medical intervention for a diagnosed disease in a given subject in a given study. A single research subject may have multiple treatments for a single diagnosis, and/or different diagnoses, and different treatments, across different studies</i></i>

    
<ul>
  <li><b>id:</b> The unique identifier for this treatment of this diagnosis in this research subject</li>
  <li><b>identifier:</b> An embedded array of information that includes the originating data center and the ID the treated researchsubject had there</li>
  <li><b>treatment_type:</b> The medical intervention undertaken</li>
  <li><b>treatment_outcome:</b> The result of the medical intervention</li>
  <li><b>days_to_treatment_start:</b> </li>
  <li><b>days_to_treatment_end:</b> </li>
  <li><b>therapeutic_agent:</b> What treatment or drug was used for this researchsubject</li>
  <li><b>treatment_anatomic_site:</b> The specific body location of the treatment</li>
  <li><b>treatment_effect:</b> </li>
  <li><b>treatment_end_reason:</b> </li>
  <li><b>number_of_cycles:</b> </li>
  <li><b>subject_id:</b> An identifier for the subject. Can be joined to the `id` field from subject results</li>
  <li><b>researchsubject_id:</b> An identifier for the researchsubject. Can be joined to the `id` field from researchsubject results</li>
  <li><b>researchsubject_diagnosis_id:</b> An identifier for the diagnosis. Can be joined to the `id` field from diagnosis results</li>
</ul>  

</div>
    
---




### specimens

We can use this same query to see what specimens are available for brain tissue at the CDA:

In [ ]:
specresults =  myquery.specimen.run()
print(specresults)

Nearly 40,000 specimens meet our search criteria! We would typically expect this number to be much larger than our number of subjects or researchsubjects. First because studies will often take more than one sample per subject, and second because any given specimen might be aliquoted out to be used in multiple tests. Since we didn't specify any further filters, our results will return all of these as seperate speciments. Let's look at a few:

In [ ]:
specresults.to_dataframe()


---

<div style="background-color:#a2f2ed;color:black;padding:20px;">

<h3>Specimen Field Definitions</h3>

<i>Any material taken as a sample from a biological entity (living or dead), or from a physical object or the environment. Specimens are usually collected as an example of their kind, often for use in some investigation.</i>

    
<ul>
  <li><b>id:</b> The unique identifier for this specimen</li>
  <li><b>identifier:</b> An embedded array of information that includes the originating data center and the ID the specimen had there</li>
  <li><b>associated_project:</b> The name of the study/project that the subject particpated in</li>
  <li><b>age_at_collection:</b> The subjects age in days (counting backwards to birth) on the day of the collection of the proximate specimen</li>
  <li><b>primary_disease_type:</b> The disease that qualifies the researchsubject for the associated_project</li>
  <li><b>anatomical_site:</b> The body part from which the proximate specimen was taken</li>
  <li><b>source_material_type:</b> The general kind of material from which the specimen was derived, indicating the physical nature of the source material</li>
  <li><b>specimen_type:</b> The high-level type of the specimen, based on its how it has been derived from the original extracted sample. One of: analyte, aliquot, portion, sample, or slide</li>
  <li><b>derived_from_specimen:</b> For derived samples, the `id` for the original sample</li>
  <li><b>subject_id:</b> An identifier for the subject. Can be joined to the `id` field from subject results</li>
  <li><b>researchsubject_id:</b> An identifier for the subject. Can be joined to the `id` field from researchsubject results</li>
</ul>  

</div>
    
---


### file

The file endpoint returns information about files that meet our search criteria, regardless of whether they are attached to subjects, research-subjects or specimens: 

In [ ]:
myquery.file.run()

In [ ]:
fileresults = myquery.file.run()
fileresults.to_dataframe()

As you might expect, searching file gives us a huge number of results. This is great if you are surveying what kind of data is available, but is less useful for getting a coherent cohort. 

A better way to get files for a specific cohort is to chain your queries together, which we cover in the next tutorial [Chaining Queries](../ChainingQueries): Combine information from multiple endpoints, and build And/Or/Like and other advanced query strings.

Another useful way to look at high level information is to use our counts feature which returns summary information rather than the full search results. Check out the [Data Summaries tutorial](../DataSummaries) to try it.



---

<div style="background-color:#a2f2ed;color:black;padding:20px;">

<h3>File Field Definitions</h3>

<i>A file is an information-bearing electronic object that contains a physical embodiment of some information using a particular character encoding.</i>

    
<ul>
  <li><b>id:</b> The unique identifier for this file</li>
  <li><b>identifier:</b> An embedded array of information that includes the originating data center and the ID the file had there</li>
  <li><b>label:</b> The full name of the file</li>
  <li><b>data_catagory:</b> A desecription of the kind of general kind data the file holds</li>
  <li><b>data_type:</b> A more specific descripton of the data type</li>
  <li><b>file_format:</b> String to identify the full file extension including compression extensions</li>
  <li><b>associated_project:</b> The name the data center uses for the study this file was generated for</li>
  <li><b>drs_uri:</b> A unique identifier that can be used to retreive this specific file from a server</li>
  <li><b>byte_size:</b> Size of the file in bytes</li>
  <li><b>checksum:</b> The md5 value for the file</li>
  <li><b>data_modality:</b> Describes the biological nature of the information gathered as the result of an activity, independent of the technology or methods used to produce the information. Always one of "Genomic", "Proteomic", or "Imaging"</li>
  <li><b>imaging_modality:</b> For files with the `data_modality` of "Imaging", a descriptor for the image type</li>
  <li><b>dbgap_accession_number:</b> The project id number for this data on dbGaP</li>
</ul>  

</div>
    
---
